In [65]:
def process_attemptreview(payload):
    try:
        payloadJson = json.loads(payload)
        jsonObject = json.loads(payloadJson.get('payload'))

        global attemptreview_df
        completed = jsonObject.get('completed', None)
        summary = jsonObject.get('summary')
        isSbs = None
        started = None
        startTime = jsonObject.get('startTime', None)
        if summary != None:
            isSbs = summary.get('stepByStepHelpEnabled')
            started = summary.get('started', None)

        attemptTreePath = jsonObject.get('attemptTreePath', None)

        attemptreview_df = attemptreview_df.append([{'type': jsonObject[u'type'],
                                         'id': jsonObject[u'id'],
                                         'scored': jsonObject[u'scored'],
                                         'cgid': jsonObject[u'cgid'],
                                         'completed': completed,
                                         'isSbs': isSbs,
                                         'started': started,
                                         'startTime': startTime,
                                         'attemptTreePath': attemptTreePath,
                                         'duedate': jsonObject['initParams'][u'dueDate']
                                         }], ignore_index=True)
    except ValueError:
        print("error")

In [61]:
def process_attempt_history(payload):
    try:
        jsonObject = json.loads(payload)
        global attempt_history_df
        completed = jsonObject.get('completed', None)
        summary = jsonObject.get('summary')
        isSbs = None
        started = None
        startTime = jsonObject.get('startTime', None)
        if summary != None:
            isSbs = summary.get('stepByStepHelpEnabled')
            started = summary.get('started', None)

        attemptTreePath = jsonObject.get('attemptTreePath', None)

        attempt_history_df = attempt_history_df.append([{'type': jsonObject[u'type'],
                                         'id': jsonObject[u'id'],
                                         'scored': jsonObject[u'scored'],
                                         'cgid': jsonObject[u'cgid'],
                                         'completed': completed,
                                         'isSbs': isSbs,
                                         'started': started,
                                         'startTime': startTime,
                                         'attemptTreePath': attemptTreePath,
                                         'duedate': jsonObject['initParams'][u'dueDate']
                                         }], ignore_index=True)
    except ValueError:
        print("error")


In [41]:
def process_componentinstance(payload):
    try:
        global componentinstance_df
        payload_as_json = json.loads(payload)
        json_object = payload_as_json['instanceJson']
        componentinstance_df = componentinstance_df.append([{'type': json_object[u'_type'],
                                       'id': json_object[u'_id']
                                       }], ignore_index=True)
    except ValueError:
        print("error")


In [55]:
def process_attempt(payload):
    try:
        jsonObject = json.loads(payload)
        global attempt_df
        completed =  jsonObject.get('completed', None)
        summary = jsonObject.get('summary')
        isSbs = None
        started = None
        startTime = jsonObject.get('startTime', None)
        if summary != None :
            isSbs = summary.get('stepByStepHelpEnabled')
            started = summary.get('started', None)
            
        attemptTreePath =  jsonObject.get('attemptTreePath', None)
        
        attempt_df = attempt_df.append([{'type':jsonObject[u'type'], 
                                'id':jsonObject[u'id'],
                                'scored':jsonObject[u'scored'],
                                'cgid':jsonObject[u'cgid'],
                                'completed':completed,
                                'isSbs' : isSbs,
                                'started' : started,
                                'startTime' : startTime,
                                'attemptTreePath' : attemptTreePath,
                                'duedate':jsonObject['initParams'][u'dueDate']
                              }], ignore_index=True)
    except ValueError:
        print("error")

In [75]:
def process_interaction(payload):
    try:
        jsonObject = json.loads(payload)
        global interaction_df
        interaction_df = interaction_df.append([{'attemptId': jsonObject[u'attemptId'],
                                                 'id': jsonObject[u'id']
                                                 }], ignore_index=True)
    except ValueError:
        print("error")


In [80]:
import pandas as pd
import json
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
import pydash

componentinstance_df = pd.DataFrame()
attempt_df = pd.DataFrame()
attempt_history_df = pd.DataFrame()
attemptreview_df = pd.DataFrame()
interaction_df = pd.DataFrame()


def process_table(table_name, session, pd):
    select_query = 'select * from {};'.format(table_name)
    print(select_query)
    rows = session.execute(select_query)
    df = pd.DataFrame(list(rows))
    if table_name == 'componentinstance':
        df.payload.apply(process_componentinstance)
    elif table_name == 'attempt':
        df.payload.apply(process_attempt)
    elif table_name == 'attempt_history':
        df.payload.apply(process_attempt_history)
    elif table_name == 'attemptreview':
        df.payload.apply(process_attemptreview)
    elif table_name == 'interaction':
        df.payload.apply(process_interaction)


def loop_all_tables():
    ap = PlainTextAuthProvider(username='cassandra', password='cassandra')
    cluster = Cluster(protocol_version=3, auth_provider=ap)
    session = cluster.connect('cas_localhost')
#     process_table('componentinstance', session, pd)
#     process_table('attempt_history', session, pd)
#     process_table('attemptreview', session, pd)
#     process_table('attempt', session, pd)
    process_table('interaction', session, pd)



loop_all_tables()
# componentinstance_df
# attempt_df
# attempt_history_df
# attemptreview_df
interaction_df


select * from interaction;


,attemptId,id
0,8e5958d4-015f-1000-0003-2cd1b8364cc8,94a46738-015f-1000-0000-92049f0c3558
